In [1]:
from huggingface_hub import login
login()

In [3]:
!pip install bitsandbytes

In [4]:
import bitsandbytes
import peft
import transformers
import datasets
import torch

In [5]:
from transformers import AutoModelForCausalLM, AutoTokenizer,BitsAndBytesConfig

model_id = 'google/gemma-3-270m-it'

bnb_config = BitsAndBytesConfig(load_in_4bit = True,
                                bnb_4bit_compute_dtype=torch.float16,
                                bnb_4bit_quant_type='nf4',
                                bnb_4bit_use_double_quant=True)

model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config = bnb_config, device_map='auto',attn_implementation='eager')

tokenizer = AutoTokenizer.from_pretrained(model_id)

In [6]:
from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(r = 8,
                         lora_alpha = 16,
                         lora_dropout= 0.05,
                         target_modules = ['q_proj','v_proj'],
                         bias='none',
                         task_type='CAUSAL_LM')

model = get_peft_model(model, lora_config)

In [7]:
from datasets import load_dataset
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('google/gemma-3-270m-it')

dataset = load_dataset('Trendyol/Trendyol-Cybersecurity-Instruction-Tuning-Dataset')
#,split='train[:50%]'

dataset = dataset.filter(lambda row: row['assistant'] is not None and row['system'] is not None and row['user'] is not None)

dataset = dataset['train'].train_test_split(0.2)

def format(example):
  system = example['system']
  user = example['user']
  assistant = example['assistant']

  prompt = f'<bos><start_of_turn>user\n{system.strip()}\n\n{user.strip()}<end_of_turn>\n'

  response = f'<start_of_turn>model\n{assistant.strip()}<end_of_turn>\n'

  return {'prompt':prompt,'response':response} #return a dict so that HF map function can work

dataset = dataset.map(format)

Map:   0%|          | 0/42559 [00:00<?, ? examples/s]

Map:   0%|          | 0/10640 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['system', 'user', 'assistant', 'prompt', 'response'],
        num_rows: 42559
    })
    test: Dataset({
        features: ['system', 'user', 'assistant', 'prompt', 'response'],
        num_rows: 10640
    })
})

In [8]:
def tokenization(example):

    input = tokenizer(example['prompt'],add_special_tokens=False,truncation=True, max_length=1024)

    output = tokenizer(example['response'], add_special_tokens=False,truncation=True, max_length=1024)

    labels = [-100] * len(input['input_ids']) + output['input_ids']

    input_ids = input['input_ids'] + output['input_ids']

    attention_mask = input['attention_mask'] + output['attention_mask']

    return {'input_ids':input_ids, 'labels':labels, 'attention_mask':attention_mask}

dataset = dataset.map(tokenization)

Map:   0%|          | 0/42559 [00:00<?, ? examples/s]

Map:   0%|          | 0/10640 [00:00<?, ? examples/s]

In [8]:
# def tokenization(example):
#     input_prompt = tokenizer(
#         example['prompt'],
#         add_special_tokens=False,
#         truncation=True,
#         max_length=1024,
#         return_tensors=None
#     )
#     input_response = tokenizer(
#         example['response'],
#         add_special_tokens=False,
#         truncation=True,
#         max_length=1024,
#         return_tensors=None
#     )

#     input_ids = input_prompt['input_ids'] + input_response['input_ids']
#     attention_mask = input_prompt['attention_mask'] + input_response['attention_mask']

#     # flatten labels: prompt masked, response real tokens
#     labels = [-100] * len(input_prompt['input_ids']) + list(input_response['input_ids'])

#     return {
#         'input_ids': list(input_ids),
#         'labels': list(labels),
#         'attention_mask': list(attention_mask)
#     }

# dataset = dataset.map(tokenization)

Map:   0%|          | 0/42559 [00:00<?, ? examples/s]

Map:   0%|          | 0/10640 [00:00<?, ? examples/s]

In [9]:
from transformers import Trainer, TrainingArguments, DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,
    padding=True,
    return_tensors = 'pt'
)

training_args = TrainingArguments(
    output_dir="./gemma3-270m-finetuned",
    overwrite_output_dir=True,
    num_train_epochs=2,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=2,
    warmup_steps=100,
    logging_steps=10,
    save_steps=500,
    eval_strategy="steps",
    eval_steps=500,
    learning_rate=5e-5,
    weight_decay=0.01,
    fp16=True,
    push_to_hub=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset['train'],
    eval_dataset=dataset['test'],
    data_collator=data_collator,
)

trainer.train()

trainer.save_model("./gemma3-270m-finetuned")
tokenizer.save_pretrained("./gemma3-270m-finetuned")

wandb: Currently logged in as: abdullahrashid (abdullahrashid-student) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss,Validation Loss


OutOfMemoryError: CUDA out of memory. Tried to allocate 3.46 GiB. GPU 0 has a total capacity of 14.74 GiB of which 3.32 GiB is free. Process 47506 has 11.42 GiB memory in use. Of the allocated memory 9.10 GiB is allocated by PyTorch, and 2.18 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
#evaluation